In [255]:
# Import dependencies and setup
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as matplotlib
import matplotlib.dates as matplotlibdates
from matplotlib.dates import date2num
import datetime

In [256]:
# Read individual csv files and store as DataFrames

atkinson_path = "Resources/dam_levels/atkinson.csv"
atkinson_df = pd.read_csv(atkinson_path)

baroon_pocket_path = "Resources/dam_levels/baroon_pocket.csv"
baroon_pocket_df = pd.read_csv(baroon_pocket_path)

bill_gunn_path = "Resources/dam_levels/bill_gunn.csv"
bill_gunn_df = pd.read_csv(bill_gunn_path)

borumba_path = "Resources/dam_levels/borumba.csv"
borumba_df = pd.read_csv(borumba_path)

cedar_pocket_path = "Resources/dam_levels/cedar_pocket.csv"
cedar_pocket_df = pd.read_csv(cedar_pocket_path)

clarendon_path = "Resources/dam_levels/clarendon.csv"
clarendon_df = pd.read_csv(clarendon_path)

cooloolabin_path = "Resources/dam_levels/cooloolabin.csv"
cooloolabin_df = pd.read_csv(cooloolabin_path)

enoggera_path = "Resources/dam_levels/enoggera.csv"
enoggera_df = pd.read_csv(enoggera_path)

ewen_maddock_path = "Resources/dam_levels/ewen_maddock.csv"
ewen_maddock_df = pd.read_csv(ewen_maddock_path)

gold_creek_path = "Resources/dam_levels/gold_creek.csv"
gold_creek_df = pd.read_csv(gold_creek_path)

hinze_path = "Resources/dam_levels/hinze.csv"
hinze_df = pd.read_csv(hinze_path)

lake_kurwongbah_path = "Resources/dam_levels/lake_kurwongbah.csv"
lake_kurwongbah_df = pd.read_csv(lake_kurwongbah_path)

lake_macdonald_path = "Resources/dam_levels/lake_macdonald.csv"
lake_macdonald_df = pd.read_csv(lake_macdonald_path)

lake_manchester_path = "Resources/dam_levels/lake_manchester.csv"
lake_manchester_df = pd.read_csv(lake_manchester_path)

leslie_harrison_path = "Resources/dam_levels/leslie_harrison.csv"
leslie_harrison_df = pd.read_csv(leslie_harrison_path)

little_nerang_path = "Resources/dam_levels/little_nerang.csv"
little_nerang_df = pd.read_csv(little_nerang_path)

maroon_path = "Resources/dam_levels/maroon.csv"
maroon_df = pd.read_csv(maroon_path)

nindooinbah_path = "Resources/dam_levels/nindooinbah.csv"
nindooinbah_df = pd.read_csv(nindooinbah_path)

north_pine_path = "Resources/dam_levels/north_pine.csv"
north_pine_df = pd.read_csv(north_pine_path)

poona_path = "Resources/dam_levels/poona.csv"
poona_df = pd.read_csv(poona_path)

seq_water_grid_path = "Resources/dam_levels/seq_water_grid.csv"
seq_water_grid_df = pd.read_csv(seq_water_grid_path)

somerset_path = "Resources/dam_levels/somerset.csv"
somerset_df = pd.read_csv(somerset_path)

wappa_path = "Resources/dam_levels/wappa.csv"
wappa_df = pd.read_csv(wappa_path)

wivenhoe_path = "Resources/dam_levels/wivenhoe.csv"
wivenhoe_df = pd.read_csv(wivenhoe_path)

wyaralong_path = "Resources/dam_levels/wyaralong.csv"
wyaralong_df = pd.read_csv(wyaralong_path)

alderley_rainfall_path = "Resources/rainfall/alderley_rainfall.csv"
alderley_df = pd.read_csv(alderley_rainfall_path)

brisbane_rainfall_path = "Resources/rainfall/brisbane_rainfall.csv"
brisbane_df = pd.read_csv(brisbane_rainfall_path)

dam_locations_path = "Resources/dam_locations.csv"
dam_locations_df = pd.read_csv(dam_locations_path)

In [257]:
# Merge csvs and store as a DataFrame

dam_frames = [atkinson_df, baroon_pocket_df, cedar_pocket_df, clarendon_df, cooloolabin_df,
                          enoggera_df, ewen_maddock_df, gold_creek_df, hinze_df, lake_kurwongbah_df,
                          lake_macdonald_df, lake_manchester_df, leslie_harrison_df, little_nerang_df,
                          maroon_df, nindooinbah_df, north_pine_df, poona_df, seq_water_grid_df, somerset_df,
                          wappa_df, wivenhoe_df, wyaralong_df]
merged_dam_df = pd.concat(dam_frames)
merged_dam_df

,Name,Date,Last Observation (%),Last Observation (ML)
0,Atkinson,4/03/17,5.4,1655.39
1,Atkinson,5/03/17,5.4,1655.39
2,Atkinson,6/03/17,5.4,1655.39
3,Atkinson,7/03/17,5.4,1655.39
4,Atkinson,8/03/17,5.4,1655.39
...,...,...,...,...
1822,Wyaralong,28-02-2022,112.4,115681.40
1823,Wyaralong,01-03-2022,105.5,108555.70
1824,Wyaralong,02-03-2022,103.1,106093.79
1825,Wyaralong,03-03-2022,105.8,108820.61


In [264]:
# Define each dam name from 'Name' column

atkinson_df = merged_dam_df[merged_dam_df.Name == 'Atkinson']
baroon_pocket_df = merged_dam_df[merged_dam_df.Name == 'Baroon Pocket']
cedar_pocket_df = merged_dam_df[merged_dam_df.Name == 'Cedar Pocket']
clarendon_df = merged_dam_df[merged_dam_df.Name == 'Clarendon']
cooloolabin_df = merged_dam_df[merged_dam_df.Name == 'Cooloolabin']
enoggera_df = merged_dam_df[merged_dam_df.Name == 'Enoggera']
ewen_maddock_df = merged_dam_df[merged_dam_df.Name == 'Ewen Maddock']
gold_creek_df = merged_dam_df[merged_dam_df.Name == 'Gold Creek']
hinze_df = merged_dam_df[merged_dam_df.Name == 'Hinze']
lake_kurwongbah_df = merged_dam_df[merged_dam_df.Name == 'Lake Kurwongbah']
lake_macdonald_df = merged_dam_df[merged_dam_df.Name == 'Lake Macdonald']
lake_manchester_df = merged_dam_df[merged_dam_df.Name == 'Lake Manchester']
leslie_harrison_df = merged_dam_df[merged_dam_df.Name == 'Leslie Harrison']
little_nerang_df = merged_dam_df[merged_dam_df.Name == 'Little Nerang']
maroon_df = merged_dam_df[merged_dam_df.Name == 'Maroon']
nindooinbah_df = merged_dam_df[merged_dam_df.Name == 'Nindooinbah']
north_pine_df = merged_dam_df[merged_dam_df.Name == 'North Pine']
poona_df = merged_dam_df[merged_dam_df.Name == 'Poona']
seq_water_grid_df = merged_dam_df[merged_dam_df.Name == 'SEQ Water Grid']
somerset_df = merged_dam_df[merged_dam_df.Name == 'Somerset']
wappa_df = merged_dam_df[merged_dam_df.Name == 'Wappa']
wivenhoe_df = merged_dam_df[merged_dam_df.Name == 'Wivenhoe']
wyaralong_df = merged_dam_df[merged_dam_df.Name == 'Wyaralong']

atkinson_df

,Name,Date,Last Observation (%),Last Observation (ML)
0,Atkinson,2017-04-03,5.4,1655.39
1,Atkinson,2017-05-03,5.4,1655.39
2,Atkinson,2017-06-03,5.4,1655.39
3,Atkinson,2017-07-03,5.4,1655.39
4,Atkinson,2017-08-03,5.4,1655.39
...,...,...,...,...
1816,Atkinson,2022-02-28,107.5,32686.23
1817,Atkinson,2022-01-03,104.1,31656.60
1818,Atkinson,2022-02-03,102.7,31223.76
1819,Atkinson,2022-03-03,103.0,31324.52


In [266]:
# Convert 'Date' columns in all .csvs to datetime

merged_dam_df['Date'] = pd.to_datetime(merged_dam_df['Date'])
merged_dam_df

,Name,Date,Last Observation (%),Last Observation (ML)
0,Atkinson,2017-04-03,5.4,1655.39
1,Atkinson,2017-05-03,5.4,1655.39
2,Atkinson,2017-06-03,5.4,1655.39
3,Atkinson,2017-07-03,5.4,1655.39
4,Atkinson,2017-08-03,5.4,1655.39
...,...,...,...,...
1822,Wyaralong,2022-02-28,112.4,115681.40
1823,Wyaralong,2022-01-03,105.5,108555.70
1824,Wyaralong,2022-02-03,103.1,106093.79
1825,Wyaralong,2022-03-03,105.8,108820.61


In [260]:
# Calculate total 'Last Observation (ML)' for each dam

atkinson_total_ML = atkinson_df['Last Observation (ML)'].sum()
baroon_pocket_total_ML = baroon_pocket_df['Last Observation (ML)'].sum()
cedar_pocket_total_ML = cedar_pocket_df['Last Observation (ML)'].sum()
clarendon_total_ML = clarendon_df['Last Observation (ML)'].sum()
cooloolabin_total_ML = cooloolabin_df['Last Observation (ML)'].sum()
enoggera_total_ML = enoggera_df['Last Observation (ML)'].sum()
ewen_maddock_total_ML = ewen_maddock_df['Last Observation (ML)'].sum()
gold_creek_total_ML = gold_creek_df['Last Observation (ML)'].sum()
hinze_total_ML = hinze_df['Last Observation (ML)'].sum()
lake_kurwongbah_total_ML = lake_kurwongbah_df['Last Observation (ML)'].sum()
lake_macdonald_total_ML = lake_macdonald_df['Last Observation (ML)'].sum()
lake_manchester_total_ML = lake_manchester_df['Last Observation (ML)'].sum()
leslie_harrison_total_ML = leslie_harrison_df['Last Observation (ML)'].sum()
little_nerang_total_ML = little_nerang_df['Last Observation (ML)'].sum()
maroon_total_ML = maroon_df['Last Observation (ML)'].sum()
nindooinbah_total_ML = nindooinbah_df['Last Observation (ML)'].sum()
north_pine_total_ML = north_pine_df['Last Observation (ML)'].sum()
poona_total_ML = poona_df['Last Observation (ML)'].sum()
seq_water_grid_total_ML = seq_water_grid_df['Last Observation (ML)'].sum()
somerset_total_ML = somerset_df['Last Observation (ML)'].sum()
wappa_total_ML = wappa_df['Last Observation (ML)'].sum()
wivenhoe_total_ML = wivenhoe_df['Last Observation (ML)'].sum()
wyaralong_total_ML = wyaralong_df['Last Observation (ML)'].sum()

atkinson_total_ML

4296619.85

In [261]:
# Calculate average of 'Last Observation (ML)' for each dam + total (merged_dam_df)

merged_avg_ML = merged_dam_df['Last Observation (ML)'].mean()

atkinson_avg_ML = atkinson_df['Last Observation (ML)'].mean()
baroon_pocket_avg_ML = baroon_pocket_df['Last Observation (ML)'].mean()
cedar_pocket_avg_ML = cedar_pocket_df['Last Observation (ML)'].mean()
clarendon_avg_ML = clarendon_df['Last Observation (ML)'].mean()
cooloolabin_avg_ML = cooloolabin_df['Last Observation (ML)'].mean()
enoggera_avg_ML = enoggera_df['Last Observation (ML)'].mean()
ewen_maddock_avg_ML = ewen_maddock_df['Last Observation (ML)'].mean()
gold_creek_avg_ML = gold_creek_df['Last Observation (ML)'].mean()
hinze_avg_ML = hinze_df['Last Observation (ML)'].mean()
lake_kurwongbah_avg_ML = lake_kurwongbah_df['Last Observation (ML)'].mean()
lake_macdonald_avg_ML = lake_macdonald_df['Last Observation (ML)'].mean()
lake_manchester_avg_ML = lake_manchester_df['Last Observation (ML)'].mean()
leslie_harrison_avg_ML = leslie_harrison_df['Last Observation (ML)'].mean()
little_nerang_avg_ML = little_nerang_df['Last Observation (ML)'].mean()
maroon_avg_ML = maroon_df['Last Observation (ML)'].mean()
nindooinbah_avg_ML = nindooinbah_df['Last Observation (ML)'].mean()
north_pine_avg_ML = north_pine_df['Last Observation (ML)'].mean()
poona_avg_ML = poona_df['Last Observation (ML)'].mean()
seq_water_grid_avg_ML = seq_water_grid_df['Last Observation (ML)'].mean()
somerset_avg_ML = somerset_df['Last Observation (ML)'].mean()
wappa_avg_ML = wappa_df['Last Observation (ML)'].mean()
wivenhoe_avg_ML = wivenhoe_df['Last Observation (ML)'].mean()
wyaralong_avg_ML = wyaralong_df['Last Observation (ML)'].mean()

merged_avg_ML

142685.1639442659

In [262]:
# Calculate average of 'Last Observation (%)' for each dam + total (merged_dam_df)

merged_avg_percent = merged_dam_df['Last Observation (%)'].mean()

atkinson_avg_percent = atkinson_df['Last Observation (%)'].mean()
baroon_pocket_avg_percent = baroon_pocket_df['Last Observation (%)'].mean()
cedar_pocket_avg_percent = cedar_pocket_df['Last Observation (%)'].mean()
clarendon_avg_percent = clarendon_df['Last Observation (%)'].mean()
cooloolabin_avg_percent = cooloolabin_df['Last Observation (%)'].mean()
enoggera_avg_percent = enoggera_df['Last Observation (%)'].mean()
ewen_maddock_avg_percent = ewen_maddock_df['Last Observation (%)'].mean()
gold_creek_avg_percent = gold_creek_df['Last Observation (%)'].mean()
hinze_avg_percent = hinze_df['Last Observation (%)'].mean()
lake_kurwongbah_avg_percent = lake_kurwongbah_df['Last Observation (%)'].mean()
lake_macdonald_avg_percent = lake_macdonald_df['Last Observation (%)'].mean()
lake_manchester_avg_percent = lake_manchester_df['Last Observation (%)'].mean()
leslie_harrison_avg_percent = leslie_harrison_df['Last Observation (%)'].mean()
little_nerang_avg_percent = little_nerang_df['Last Observation (%)'].mean()
maroon_avg_percent = maroon_df['Last Observation (%)'].mean()
nindooinbah_avg_percent = nindooinbah_df['Last Observation (%)'].mean()
north_pine_avg_percent = north_pine_df['Last Observation (%)'].mean()
poona_avg_percent = poona_df['Last Observation (%)'].mean()
seq_water_grid_avg_percent = seq_water_grid_df['Last Observation (%)'].mean()
somerset_avg_percent = somerset_df['Last Observation (%)'].mean()
wappa_avg_percent = wappa_df['Last Observation (%)'].mean()
wivenhoe_avg_percent = wivenhoe_df['Last Observation (%)'].mean()
wyaralong_avg_percent = wyaralong_df['Last Observation (%)'].mean()

merged_avg_percent

88.98479695129198

In [281]:
# Group by year and calculate yearly totals

merged_dam_yearly_df = merged_dam_df.groupby(merged_dam_df['Date'].dt.to_period('Y')).sum()

atkinson_yearly_df = atkinson_df.groupby(atkinson_df['Date'].dt.to_period('Y')).sum()
baroon_pocket_yearly_df = baroon_pocket_df.groupby(baroon_pocket_df['Date'].dt.to_period('Y')).sum()
cedar_pocket_yearly_df = cedar_pocket_df.groupby(cedar_pocket_df['Date'].dt.to_period('Y')).sum()
clarendon_yearly_df = clarendon_df.groupby(clarendon_df['Date'].dt.to_period('Y')).sum()
cooloolabin_yearly_df = cooloolabin_df.groupby(cooloolabin_df['Date'].dt.to_period('Y')).sum()
enoggera_yearly_df = enoggera_df.groupby(enoggera_df['Date'].dt.to_period('Y')).sum()
ewen_maddock_yearly_df = ewen_maddock_df.groupby(ewen_maddock_df['Date'].dt.to_period('Y')).sum()
gold_creek_yearly_df = gold_creek_df.groupby(gold_creek_df['Date'].dt.to_period('Y')).sum()
hinze_yearly_df = hinze_df.groupby(hinze_df['Date'].dt.to_period('Y')).sum()
lake_kurwongbah_yearly_df = lake_kurwongbah_df.groupby(lake_kurwongbah_df['Date'].dt.to_period('Y')).sum()
lake_macdonald_yearly_df = lake_macdonald_df.groupby(lake_macdonald_df['Date'].dt.to_period('Y')).sum()
lake_manchester_yearly_df = lake_manchester_df.groupby(lake_manchester_df['Date'].dt.to_period('Y')).sum()
leslie_harrison_yearly_df = leslie_harrison_df.groupby(leslie_harrison_df['Date'].dt.to_period('Y')).sum()
little_nerang_yearly_df = little_nerang_df.groupby(little_nerang_df['Date'].dt.to_period('Y')).sum()
maroon_yearly_df = maroon_df.groupby(maroon_df['Date'].dt.to_period('Y')).sum()
nindooinbah_yearly_df = nindooinbah_df.groupby(nindooinbah_df['Date'].dt.to_period('Y')).sum()
north_pine_yearly_df = north_pine_df.groupby(north_pine_df['Date'].dt.to_period('Y')).sum()
poona_yearly_df = poona_df.groupby(poona_df['Date'].dt.to_period('Y')).sum()
seq_water_grid_yearly_df = seq_water_grid_df.groupby(seq_water_grid_df['Date'].dt.to_period('Y')).sum()
somerset_yearly_df = somerset_df.groupby(somerset_df['Date'].dt.to_period('Y')).sum()
wappa_yearly_df = wappa_df.groupby(wappa_df['Date'].dt.to_period('Y')).sum()
wivenhoe_yearly_df = wivenhoe_df.groupby(wivenhoe_df['Date'].dt.to_period('Y')).sum()
wyaralong_yearly_df = wyaralong_df.groupby(wyaralong_df['Date'].dt.to_period('Y')).sum()

merged_dam_yearly_df = merged_dam_yearly_df.reset_index(drop=False)
merged_dam_yearly_df

,Date,Last Observation (%),Last Observation (ML)
0,2017,556649.1,1.060506e+09
1,2018,703817.8,1.333415e+09
2,2019,636437.6,1.132013e+09
3,2020,630635.8,1.051705e+09
4,2021,1077507.1,1.194358e+09
5,2022,130979.3,2.186398e+08


In [290]:
# Group by month and calculate monthly totals

merged_dam_monthly_df = merged_dam_df.groupby(pd.Grouper(key='Date', freq='1M')).sum()
merged_dam_monthly_df.index = merged_dam_monthly_df.index.strftime('%B')
merged_dam_monthly_df = merged_dam_monthly_df.reset_index(drop=False)

january = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'January']
february = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'February']
march = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'March']
april = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'April']
may = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'May']
june = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'June']
july = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'July']
august = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'August']
september = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'September']
october = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'October']
november = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'November']
december = merged_dam_monthly_df[merged_dam_monthly_df.Date == 'December']

# months = [january, february, march, april, may, june, july, august, september, october, november, december]

# merged_dam_df = merged_dam_df.groupby(merged_dam_df['Date']).aggregate(months)





# merged_dam_monthly_df = pd.concat(months)

# merged_dam_monthly_df = merged_dam_df.groupby(merged_dam_df['Date'].dt.to_period('m')).sum()

# atkinson_monthly_df = atkinson_df.groupby(atkinson_df['Date'].dt.to_period('m')).sum()
# baroon_pocket_yearly_df = baroon_pocket_df.groupby(baroon_pocket_df['Date'].dt.to_period('Y')).sum()
# cedar_pocket_yearly_df = cedar_pocket_df.groupby(cedar_pocket_df['Date'].dt.to_period('Y')).sum()
# clarendon_yearly_df = clarendon_df.groupby(clarendon_df['Date'].dt.to_period('Y')).sum()
# cooloolabin_yearly_df = cooloolabin_df.groupby(cooloolabin_df['Date'].dt.to_period('Y')).sum()
# enoggera_yearly_df = enoggera_df.groupby(enoggera_df['Date'].dt.to_period('Y')).sum()
# ewen_maddock_yearly_df = ewen_maddock_df.groupby(ewen_maddock_df['Date'].dt.to_period('Y')).sum()
# gold_creek_yearly_df = gold_creek_df.groupby(gold_creek_df['Date'].dt.to_period('Y')).sum()
# hinze_yearly_df = hinze_df.groupby(hinze_df['Date'].dt.to_period('Y')).sum()
# lake_kurwongbah_yearly_df = lake_kurwongbah_df.groupby(lake_kurwongbah_df['Date'].dt.to_period('Y')).sum()
# lake_macdonald_yearly_df = lake_macdonald_df.groupby(lake_macdonald_df['Date'].dt.to_period('Y')).sum()
# lake_manchester_yearly_df = lake_manchester_df.groupby(lake_manchester_df['Date'].dt.to_period('Y')).sum()
# leslie_harrison_yearly_df = leslie_harrison_df.groupby(leslie_harrison_df['Date'].dt.to_period('Y')).sum()
# little_nerang_yearly_df = little_nerang_df.groupby(little_nerang_df['Date'].dt.to_period('Y')).sum()
# maroon_yearly_df = maroon_df.groupby(maroon_df['Date'].dt.to_period('Y')).sum()
# nindooinbah_yearly_df = nindooinbah_df.groupby(nindooinbah_df['Date'].dt.to_period('Y')).sum()
# north_pine_yearly_df = north_pine_df.groupby(north_pine_df['Date'].dt.to_period('Y')).sum()
# poona_yearly_df = poona_df.groupby(poona_df['Date'].dt.to_period('Y')).sum()
# seq_water_grid_yearly_df = seq_water_grid_df.groupby(seq_water_grid_df['Date'].dt.to_period('Y')).sum()
# somerset_yearly_df = somerset_df.groupby(somerset_df['Date'].dt.to_period('Y')).sum()
# wappa_yearly_df = wappa_df.groupby(wappa_df['Date'].dt.to_period('Y')).sum()
# wivenhoe_yearly_df = wivenhoe_df.groupby(wivenhoe_df['Date'].dt.to_period('Y')).sum()
# wyaralong_yearly_df = wyaralong_df.groupby(wyaralong_df['Date'].dt.to_period('Y')).sum()

february

,Date,Last Observation (%),Last Observation (ML)
1,February,16782.8,3.172531e+07
13,February,54198.8,1.005487e+08
25,February,49666.2,9.065467e+07
37,February,53430.1,8.745582e+07
49,February,48300.4,7.534325e+07
61,February,41516.7,7.161446e+07


In [24]:
# Calculate averages by month

In [ ]:
# Calculate averages by decade